# DHTML : Dynamic HTML

#### 일반적인 HTML
- Page Reloading 
- request를 보내면 무조건 응답을 보냄


#### DHTML 
- AJAX : Asynchronous Javascript AND XML
- 기존 dom에 데이터를 뒤집어 씌움 (화면 깜빡임으로 판단)
- request를 최초에만 받음
- Automation

In [4]:
import requests
import time

def download(method, url, 
             param = None, data = None, 
            timeout = 1,maxretries = 3):
    
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}
    #headers = None
    
    try:
        resp = requests.request(method, url,
                                params= param,
                                data=data, headers=headers)
        resp.raise_for_status()
        
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries > 0: 
            time.sleep(timeout) # param에따라 몇초 기다릴지 결정 할 수가 있다.
            print(maxretries) # 재귀적으로 자기 자신을 부르게 코드를 짜면 된다.
            resp = download(method,
                            url, param = param, data=data,
                            timeout=timeout,
                            maxretries = maxretries - 1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

In [5]:
def geturl(urlsrc):
    if '?' in urlsrc:
        return urlsrc.split('?')[0]+'?'
    else:
        return urlsrc

def getparams(urlsrc):
    if '?' in  urlsrc:
        paramList = urlsrc.split('?')[1].split('&')
        params = {_.split('=')[0]:compat.unquote(_.split('=')[1]) for _ in paramList}
        return params
    else:
        return None

In [6]:
from bs4 import BeautifulSoup
from requests import compat

In [7]:
import json

In [24]:
url = "http://example.webscraping.com/places/ajax/search.json?&search_term=korea&page_size=10&page=0"
#param = getparams(url)
urlsrc = geturl(url)

params = {
    'search_term' : 'korea',
    'page_size' : 10,
    'page' : 0
}

html = download('get',urlsrc,params)
dom = BeautifulSoup(html.text,"lxml")

In [18]:
url = "http://example.webscraping.com/places/ajax/search.json?&search_term=korea&page_size=10&page=0"

In [25]:
print(html.text) # 이런건 원래 form을 찾아라.
# 두번째 : 누를때마다 network에 내가 한 행동이 뜬다.

{"records": [{"pretty_link": "<div><a href=\"/places/default/view/North-Korea-165\"><img src=\"/places/static/images/flags/kp.png\" /> North Korea</a></div>", "country": "North Korea", "id": 4552797}, {"pretty_link": "<div><a href=\"/places/default/view/South-Korea-211\"><img src=\"/places/static/images/flags/kr.png\" /> South Korea</a></div>", "country": "South Korea", "id": 4552843}], "num_pages": 1, "error": ""}



In [176]:
%%writefile interP.json
{
    "id":"",
    "pw":""
}

Overwriting evt.json


In [177]:
import json

In [38]:
with open('evt.json') as fp:
    account = json.load(fp)

In [39]:
account["id"]

'http94'

In [52]:
url = 'https://everytime.kr/login'
url = requests.compat.urljoin(url,'/user/login') # 왜 이거인지는 페이지 소스보면 나온다.
data= {
    "userid":account["id"],
    "password":account["pw"]
}
html = download("post",url,data=data)
print(html.text)

<!DOCTYPE html>
<html lang="ko">
<head>
  <title>에브리타임</title>
  <meta charset="utf-8">
  <meta name="referrer" content="origin">
  <meta http-equiv="X-UA-Compatible" content="IE=edge">
  <meta property="fb:app_id" content="258676027581965">
  <meta property="og:type" content="website">
  <meta property="og:image" content="https://everytime.kr/images/og_image.png">
  <meta property="og:url" content="https://everytime.kr/">
  <meta property="og:site_name" content="에브리타임">
  <meta property="og:title" content="에브리타임">
  <meta property="og:description" content="전국 400개 대학을 지원하는 대학교 커뮤니티 및 시간표 서비스. 시간표 작성 및 학업 관리, 학교 생활 정보, 학교별 익명 커뮤니티 기능을 제공합니다.">
  <meta name="description" content="전국 400개 대학을 지원하는 대학교 커뮤니티 및 시간표 서비스. 시간표 작성 및 학업 관리, 학교 생활 정보, 학교별 익명 커뮤니티 기능을 제공합니다.">
  <meta name="keywords" content="에브리타임, 에타, everytime, 시간표, 수강신청, 강의평가, 학점계산기, 학식, 오늘의학식, 책방, 중고책, 대학생, 대학교, 대학, 대학생 시간표, 대학교 시간표, 대학생 커뮤니티, 대학교 커뮤니티, 시간표 앱, 시간표 어플">
  <meta name="naver-site-verification" content="736673837

In [53]:
session = requests.Session() # 세션과 쿠키를 관리해줌

In [54]:
html = session.post(url,data) # 최초접속, 쿠키는 미생성

In [55]:
session.cookies # 단일페이지 로그인 처리일 때만 이런 방식이 통함(소규모 사이트)

<RequestsCookieJar[Cookie(version=0, name='etsid', value='s%3AZXd2mut_ILsaDT51aXbStPLevEwGe_he.WFjNTYqdvEwRkE9DrbZYK3jdqGTjx7vgKL5%2BpdTQa14', port=None, port_specified=False, domain='.everytime.kr', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)]>

In [56]:
url = 'https://everytime.kr/find/board/article/list'
data = {
    'id': 370438,
'limit_num': 20,
'start_num': 0,
'moiminfo': 'true'
}

#html = download('post',url,data=data)

In [57]:
html = session.post(url,data=data)

In [43]:
from lxml import etree

In [58]:
res = etree.dump(html.text)

TypeError: Argument 'elem' has incorrect type (expected lxml.etree._Element, got str)

In [59]:
html.text

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n  <moim id="370438" name="자유게시판" info="" picture="" type="2" layout="21" priv_anonym="0" priv_comment_anonym="0" is_primary="1" is_schoolcommunity="1" is_secret="0" is_manageable="0" is_searchable="1" is_member="0" join_to_write="0" join_to_comment="0" auth_to_write="0" auth_to_comment="0" placeholder="여기를 눌러서 글을 작성할 수 있습니다. &lt;br />아래 내용을 모두 지켜서 깨끗한 에브리타임을 만들어주세요 :) &lt;br />&lt;br />[정보통신망법에 의한 홍보 게시물 작성 금지] &lt;br />1. 회사/단체와 관련된 공모전, 대외활동, 구인, 펀딩 홍보 &lt;br />2. 커뮤니티, 어플, 쇼핑몰, 카페, 블로그 홍보 및 방문 유도 &lt;br />3. 토익, 한자 등 어학원 홍보 &lt;br />4. 연극, 영화 티켓 할인 서비스 홍보 &lt;br />5. 안과, 치과, 피부과 등 의료업체 홍보 &lt;br />&lt;br />[커뮤니티 이용규칙에 어긋나는 행위 금지] &lt;br />1. 비밀게시판 등 인증 후 사용할 수 있는 게시판의 스크린샷, 게시물 내용 유출 &lt;br />2. 욕설, 비하, 음란물, 개인정보가 포함된 게시물 게시 &lt;br />3. 특정인이나 단체/지역을 비방하는 행위 &lt;br />4. 기타 현행법에 어긋나는 행위 &lt;br />&lt;br />자세한 내용은 홈 탭 우측 상단의 [내 정보] > [커뮤니티 이용규칙]을 참고하시기 바랍니다." is_quick="1" is_accessible="1" is_writable="1" is_readable="1" is_commentabl

In [285]:
xmlObj = etree.tostring(html.text)

TypeError: Type 'str' cannot be serialized.

In [284]:
xmlObj = etree.fromstring(html.text)

ValueError: Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.

In [293]:
parser = etree.XMLParser(ns_clean=True, recover=True, encoding='UTF-8')

In [60]:
a = etree.XML(html.content)

In [66]:
a

<Element response at 0x2692b9844c8>

In [73]:
xmlRoot = etree.ElementTree(xmlObj).getroot()

In [93]:
from html import escape

In [131]:
itemList = xmlRoot.findall('.//article')
for i,_ in enumerate(itemList):
    print("[",_.get('title'),"]",_.get('text'))
#type(itemList[0])
#etree._Element

[ 솔직히 연공은 분리해야지 ] 설카포의치한도 못간 쉑들이 어디 같은 캠퍼스를 쓰려고 해
[ 그냥 니네 과의 가치는 ] 니네과에서 배우는 과목을 학교 밖에서 배우기 얼마나 어렵냐로 평가하면 안됨? 경영, 경제 개 ㅎㅌㅊ 언제든지 인강으로 가능 의치대 ㅆㅅㅌㅊ 노숙
[ 11학번인데 상공전 8년후에도 하고있을 줄 몰랐네 ] 2030년에도 계속되는 각이냐?
[ 강의평가 없이 성적조회 가능? ] 제곧내다 해외에서 포탈이 더럽게 안터져서 강평 못했는데 성적조회 어떻게 해야되냐ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 아미치겠네
[ 심술쟁이 에붕이는못말려 ] 유후
[ 이정도면 반수 ㅁㅌㅊ ] 폐교된의대->연대문과  ??
[ 탈모치료제 개발 vs 대통령 ] 뭐가 낫냐
[ 교수님한테 ] 부탁드릴 일이 있어서 메일 보냈는데 읽고 답장이 없는데 꼭 부탁드릴 일이라.. 다시 정중하게 사정 말하고 답장ㄱㅣ다리고 있겠습니다. 이렇게 보내
[ 상공전 하는 ㅂㅅ들봐라 ] 자기가 행복한 삶 살면 되는거지 남 까내리고 쥐랄. 지들도 똑같으니까 자기 까내리는거에 민감하게 반응해서 남까내리지. 너네들은 멍멍 짖는 개보고
[ 애초에 수리가나형 사과탐 나눈게 문제 ] 이제 통합수능봐봐라  연공이 높을지 연경이 높을지 ㅋ  뻔한거아니겠노  연경 머가리 비어서 문과로 도망간주제에 말이많노
[ 의대다니다가 문과비상경 ] 온사람도있는대 자기하고싶은거 하는거지 하여간 여기는 참 ㅉ
[ 연세우유 new디자인 ] 전보다 낫누
[ 문과의 수문장 ] 대우관파오후들이 없으니 공찐들이 설치네 경영놈들은 애초에 에붕이가드물고
[ 연공 특 : 현실은 복전하는 문과생이 과탑 ] 컴과 과탑이 문과생이라며? ㅋㅋ
[ 변리사 vs 변호사 ] ㅈㄱㄴ
[ 재산 1000억 밴처기업가 vs 검찰총장 ] 뭐가 낫냐?
[ 연공특 ] 학창시절에 반친구들끼리 애니봄
[ 에타 병신들은 이래서 좋아 ] 나만 병신이 아니란 안도감을 줘
[ 본인 경영 공대 복전하는데 ] 경영이나 공대나 다 ㅂㅅ 내가최고
[ 으디 연경이 ] 전화기 공머생인데 애당초에 문과 풀이랑 이

In [69]:
xmlObj = etree.fromstring('\n'.join(html.text.splitlines()[1:]))

In [70]:
type(xmlObj)

lxml.etree._Element

In [75]:
etree.dump(xmlRoot)

<response>
  <moim id="370438" name="자유게시판" info="" picture="" type="2" layout="21" priv_anonym="0" priv_comment_anonym="0" is_primary="1" is_schoolcommunity="1" is_secret="0" is_manageable="0" is_searchable="1" is_member="0" join_to_write="0" join_to_comment="0" auth_to_write="0" auth_to_comment="0" placeholder="여기를 눌러서 글을 작성할 수 있습니다. &lt;br /&gt;아래 내용을 모두 지켜서 깨끗한 에브리타임을 만들어주세요 :) &lt;br /&gt;&lt;br /&gt;[정보통신망법에 의한 홍보 게시물 작성 금지] &lt;br /&gt;1. 회사/단체와 관련된 공모전, 대외활동, 구인, 펀딩 홍보 &lt;br /&gt;2. 커뮤니티, 어플, 쇼핑몰, 카페, 블로그 홍보 및 방문 유도 &lt;br /&gt;3. 토익, 한자 등 어학원 홍보 &lt;br /&gt;4. 연극, 영화 티켓 할인 서비스 홍보 &lt;br /&gt;5. 안과, 치과, 피부과 등 의료업체 홍보 &lt;br /&gt;&lt;br /&gt;[커뮤니티 이용규칙에 어긋나는 행위 금지] &lt;br /&gt;1. 비밀게시판 등 인증 후 사용할 수 있는 게시판의 스크린샷, 게시물 내용 유출 &lt;br /&gt;2. 욕설, 비하, 음란물, 개인정보가 포함된 게시물 게시 &lt;br /&gt;3. 특정인이나 단체/지역을 비방하는 행위 &lt;br /&gt;4. 기타 현행법에 어긋나는 행위 &lt;br /&gt;&lt;br /&gt;자세한 내용은 홈 탭 우측 상단의 [내 정보] &gt; [커뮤니티 이용규칙]을 참고하시기 바랍니다." is_quick="1" is_accessible="1" is_writable="1" is_readable="1" is_c

In [68]:
'\n'.join(html.text.splitlines()[1:])

'<response>\n  <moim id="370438" name="자유게시판" info="" picture="" type="2" layout="21" priv_anonym="0" priv_comment_anonym="0" is_primary="1" is_schoolcommunity="1" is_secret="0" is_manageable="0" is_searchable="1" is_member="0" join_to_write="0" join_to_comment="0" auth_to_write="0" auth_to_comment="0" placeholder="여기를 눌러서 글을 작성할 수 있습니다. &lt;br />아래 내용을 모두 지켜서 깨끗한 에브리타임을 만들어주세요 :) &lt;br />&lt;br />[정보통신망법에 의한 홍보 게시물 작성 금지] &lt;br />1. 회사/단체와 관련된 공모전, 대외활동, 구인, 펀딩 홍보 &lt;br />2. 커뮤니티, 어플, 쇼핑몰, 카페, 블로그 홍보 및 방문 유도 &lt;br />3. 토익, 한자 등 어학원 홍보 &lt;br />4. 연극, 영화 티켓 할인 서비스 홍보 &lt;br />5. 안과, 치과, 피부과 등 의료업체 홍보 &lt;br />&lt;br />[커뮤니티 이용규칙에 어긋나는 행위 금지] &lt;br />1. 비밀게시판 등 인증 후 사용할 수 있는 게시판의 스크린샷, 게시물 내용 유출 &lt;br />2. 욕설, 비하, 음란물, 개인정보가 포함된 게시물 게시 &lt;br />3. 특정인이나 단체/지역을 비방하는 행위 &lt;br />4. 기타 현행법에 어긋나는 행위 &lt;br />&lt;br />자세한 내용은 홈 탭 우측 상단의 [내 정보] > [커뮤니티 이용규칙]을 참고하시기 바랍니다." is_quick="1" is_accessible="1" is_writable="1" is_readable="1" is_commentable="1"/>\n  <hashtags/>\n  <article id="8

In [67]:
html.text

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n  <moim id="370438" name="자유게시판" info="" picture="" type="2" layout="21" priv_anonym="0" priv_comment_anonym="0" is_primary="1" is_schoolcommunity="1" is_secret="0" is_manageable="0" is_searchable="1" is_member="0" join_to_write="0" join_to_comment="0" auth_to_write="0" auth_to_comment="0" placeholder="여기를 눌러서 글을 작성할 수 있습니다. &lt;br />아래 내용을 모두 지켜서 깨끗한 에브리타임을 만들어주세요 :) &lt;br />&lt;br />[정보통신망법에 의한 홍보 게시물 작성 금지] &lt;br />1. 회사/단체와 관련된 공모전, 대외활동, 구인, 펀딩 홍보 &lt;br />2. 커뮤니티, 어플, 쇼핑몰, 카페, 블로그 홍보 및 방문 유도 &lt;br />3. 토익, 한자 등 어학원 홍보 &lt;br />4. 연극, 영화 티켓 할인 서비스 홍보 &lt;br />5. 안과, 치과, 피부과 등 의료업체 홍보 &lt;br />&lt;br />[커뮤니티 이용규칙에 어긋나는 행위 금지] &lt;br />1. 비밀게시판 등 인증 후 사용할 수 있는 게시판의 스크린샷, 게시물 내용 유출 &lt;br />2. 욕설, 비하, 음란물, 개인정보가 포함된 게시물 게시 &lt;br />3. 특정인이나 단체/지역을 비방하는 행위 &lt;br />4. 기타 현행법에 어긋나는 행위 &lt;br />&lt;br />자세한 내용은 홈 탭 우측 상단의 [내 정보] > [커뮤니티 이용규칙]을 참고하시기 바랍니다." is_quick="1" is_accessible="1" is_writable="1" is_readable="1" is_commentabl

In [39]:
html.json()["records"]

[{'pretty_link': '<div><a href="/places/default/view/North-Korea-165"><img src="/places/static/images/flags/kp.png" /> North Korea</a></div>',
  'country': 'North Korea',
  'id': 4552797},
 {'pretty_link': '<div><a href="/places/default/view/South-Korea-211"><img src="/places/static/images/flags/kr.png" /> South Korea</a></div>',
  'country': 'South Korea',
  'id': 4552843}]

In [47]:
for _ in html.json()["records"]:
    print(_["country"], _["pretty_link"])

North Korea <div><a href="/places/default/view/North-Korea-165"><img src="/places/static/images/flags/kp.png" /> North Korea</a></div>
South Korea <div><a href="/places/default/view/South-Korea-211"><img src="/places/static/images/flags/kr.png" /> South Korea</a></div>


### Selenium
- dom으로 중간에 볼 수 없는 것을 찾을 수 있게 해준다.
- 브라우저에서 하는 것을 자동화 해주는 것
- 웹 시스템을 만들면 시나리오 혹은 단위 테스트를 쉽게 해주는 것.(테스팅 목적)
- JAVAscript를 해석할 수 있게 해줌.
- 원래는 testing목적이지만, 많은 사람들이 dhtml에서 이용함.
- 에자일 방법론

In [48]:
!pip install selenium

http://chromedriver.chromium.org/downloads

In [168]:
%%writefile EveryT.json
{
    "id":"",
    "pw":""
} # json은 무조건 double quote(single은 오류남)

Writing EveryT.json


In [30]:
import json

In [31]:
with open('my.json') as fp:
    account = json.load(fp)

In [32]:
account["id"]

'http94'

In [8]:
from selenium import webdriver

driver = webdriver.Chrome()
# 창 닫고싶으면 driver.close()

In [71]:
driver.get("http://example.webscraping.com/places/default/search")

In [72]:
driver.page_source # html문서 가지고 있다.

'<html class=" js flexbox flexboxlegacy canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers no-applicationcache svg inlinesvg smil svgclippaths" lang="en-us" style=""><!--<![endif]--><head>\n<title>Example web scraping website</title>\n  <!--[if !HTML5]>\n      <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n  <![endif]-->\n  <!-- www.phpied.com/conditional-comments-block-downloads/ -->\n  <!-- Always force latest IE rendering engine\n       (even in intranet) & Chrome Frame\n       Remove this if you use the .htaccess -->\n\t   \n  <meta charset="utf-8">\n\n  <!-- http://dev.w3.org/html5/markup/meta.name.html -->\n  <meta name="applicati

In [74]:
inputTag = driver.find_element_by_id("search_term")
inputTag

<selenium.webdriver.remote.webelement.WebElement (session="d7d4850df00897ad1820d07b5f4f6da3", element="be9a80be-aa15-4734-b349-a1c5968d849f")>

In [77]:
inputTag.send_keys("korea")

In [78]:
driver.find_element_by_xpath("//input[@type='submit']").click()

In [79]:
from bs4 import BeautifulSoup
dom = BeautifulSoup(driver.page_source,"lxml")
dom.select("#results")

[<div id="results"><table><tbody><tr><td><div><a href="/places/default/view/North-Korea-165"><img src="/places/static/images/flags/kp.png"/> North Korea</a></div></td><td><div><a href="/places/default/view/South-Korea-211"><img src="/places/static/images/flags/kr.png"/> South Korea</a></div></td></tr></tbody></table></div>]

In [88]:
driver.find_element_by_css_selector("#results").find_elements()

[]

In [84]:
dir(driver.find_element_by_css_selector("#results"))
# 단, 렌더링 하기 전에 후다닥 지나가버리면 안된다.

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_execute',
 '_id',
 '_parent',
 '_upload',
 '_w3c',
 'clear',
 'click',
 'find_element',
 'find_element_by_class_name',
 'find_element_by_css_selector',
 'find_element_by_id',
 'find_element_by_link_text',
 'find_element_by_name',
 'find_element_by_partial_link_text',
 'find_element_by_tag_name',
 'find_element_by_xpath',
 'find_elements',
 'find_elements_by_class_name',
 'find_elements_by_css_selector',
 'find_elements_by_id',
 'find_elements_by_link_text',
 'find_elements_by_name',
 'find_elements_by_partial_link_text',
 'find_elements_by_tag_name',
 'find_elements_by_xpath',
 'get_attribute',
 'get_property',
 'id',
 

- find_element_by_id
- find_element_by_name
- find_element_by_xpath
- find_element_by_link_text
- find_element_by_partial_link_text
- find_element_by_tag_name
- find_element_by_class_name
- find_element_by_css_selector

In [119]:
driver.get("https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fmail.naver.com")

In [ ]:
inputTag = driver.find_element_by_class_name("ico_local_login.lang_ko").click()

In [118]:
inputTag = driver.find_element_by_xpath("//input[@type='text']")
inputTag.send_keys(account["id"])
inputTag = driver.find_element_by_xpath("//input[@type='password']")
inputTag.send_keys(account["pw"])
driver.find_element_by_xpath("//input[@type='submit']").click()

In [78]:
driver.find_element_by_xpath("//input[@type='submit']").click()

In [77]:
inputTag.send_keys("korea")

In [79]:
from bs4 import BeautifulSoup
dom = BeautifulSoup(driver.page_source,"lxml")
dom.select("#results")

[<div id="results"><table><tbody><tr><td><div><a href="/places/default/view/North-Korea-165"><img src="/places/static/images/flags/kp.png"/> North Korea</a></div></td><td><div><a href="/places/default/view/South-Korea-211"><img src="/places/static/images/flags/kr.png"/> South Korea</a></div></td></tr></tbody></table></div>]

In [148]:
for _ in driver.find_elements_by_css_selector("strong.mail_title"):
    print("[CSS]",_.text)

[CSS] 새로운 기기(브라우저)에서 로그인 되었습니다.
[CSS] [DBGuide.net] (데이터바우처) 3차수요기업 모집 찾아가는 설명회 및 매칭데이 신청안내
[CSS] [2019 데이터 청년 캠퍼스] 취업지원 프로그램 및 자격검정 신청 안내(재발송)
[CSS] 새로운 기기(브라우저)에서 로그인 되었습니다.
[CSS] 2019학년도 여름계절제 학부 강의정보 공유를 위한 설문 실시 안내
[CSS] [2019 데이터 청년 캠퍼스] 취업지원 프로그램 및 자격검정 신청 안내
[CSS] [데이터진흥원] 데이터 자격시험 응시 최종 확인
[CSS] [데이터진흥원] 데이터 자격시험 응시 수요파악
[CSS] [ 공공데이터포털 ] 회원가입을 위한 이메일 인증번호
[CSS] [ 공공데이터포털 ] 회원가입을 위한 이메일 인증번호
[CSS] [ 공공데이터포털 ] 회원가입을 위한 이메일 인증번호
[CSS] 연세대학교가 서동욱님의 성적증명서를 전송하셨습니다.
[CSS] [DBGuide.net] (2019 데이터 품질대상) 대한민국 최고의 데이터 품질 관리 기관·기업을 찾습니다!
[CSS] [데이터진흥원_빅데이터_청년인재_고려대학교_과정] 비밀번호 변경
[CSS] 특별세미나 안내
[CSS] [Y-DEC] 2019 여름방학 Adobe 교육 프로그램 안내
[CSS] CATCH 이용약관 개정안내
[CSS] [DBGuide.net] 마이데이터 아이디어 발표회에 여러분을 초대합니다.(7/11)
[CSS] [GitHub] Please verify your device
[CSS] [GitHub] @milkyway103 has invited you to join the @bigdata07 organization
[CSS] 고려대_임시_수업자료 - Invitation to view
[CSS] You've been made an owner of the bigdata201907 organization
[CSS] [GitHub] @Ju0Seo has invited you to join the @bigda

In [203]:
for _ in driver.find_elements_by_xpath("//div[@class='mTitle']//span/strong"):
    print("[CSS]",_.text)

[CSS] (광고)여행가는데 쇼핑이 빠질 수 있나요? 동화면세점 스마일페이 첫 결제 5천원 즉시할인
[CSS] 새로운 기기(브라우저)에서 로그인 되었습니다.
[CSS] 새로운 기기(브라우저)에서 로그인 되었습니다.
[CSS] 새로운 환경에서 로그인 되었습니다.
[CSS] 새로운 환경에서 로그인 되었습니다.
[CSS] (광고)누구나 5천원 적립 스마일클럽이면 두배 적립! 마켓컬리 여름에도 늘 신선 풀콜드 샛별배송!
[CSS] [Y-DEC] 2019-2학기 동아리 홍보전
[CSS] 귀하를 위한 추천 강좌
[CSS] [창업 준비를 한번에!] 2019년 국제캠퍼스 학생벤처센터 창업 원스톱 상담창구 운영 안내
[CSS] ACS Pubs Forum 참가 및 포스터 발표 신청
[CSS] 연세대학교 통일연구원 Issue Brief 제129호
[CSS] The Spirit of Yonsei (섬김과 나눔, 기부와 헌신) - 7월호
[CSS] 연세소식(2019.07.15.) 함께 뛰며 나눔의 정신 나눠요
[CSS] [특별할인] 오늘 만료되는 우버이츠 할인을 놓치지마세요!
[CSS] 장기 미이용 휴면처리 완료 메일
[CSS] 이용 약관 업데이트
[CSS] 【アダルト動画】おすすめの作品をピックアップ★
[CSS] [특별할인] 우버이츠 첫 주문 1만원 할인 받으세요!
[CSS] 새로운 환경에서 로그인 되었습니다.
[CSS] [신한카드] 결제일별 이용기간 (신용공여기간) 변경 연기 안내
[CSS] [DBGuide.net] 2019 마이데이터 서비스 오픈 컨설팅 지원 안내
[CSS] (광고) POE, 랑그릿사, 일곱개의 대죄를 하는 사람이라면 주목!
[CSS] 最優秀女優賞👑相沢みなみの待望の新作💖【一番の痴女姿見せます💋】
[CSS] 2019년도 여름학기 2차 교내특강(토익, 토플) 개강 안내
[CSS] 새로운 기기(브라우저)에서 로그인 되었습니다.
[CSS] 【アダルト動画】本日更新セール情報★
[CSS] [DBGuide.net] 마이데이터 아이디어 발표회에 여러분을 초대합니다.(7/11)
[CS

In [204]:
for _ in driver.find_elements_by_xpath("//div[@class='mTitle']//span/strong"):
    print("[CSS]",_.text)

[CSS] (광고)여행가는데 쇼핑이 빠질 수 있나요? 동화면세점 스마일페이 첫 결제 5천원 즉시할인
[CSS] 새로운 기기(브라우저)에서 로그인 되었습니다.
[CSS] 새로운 기기(브라우저)에서 로그인 되었습니다.
[CSS] 새로운 환경에서 로그인 되었습니다.
[CSS] 새로운 환경에서 로그인 되었습니다.
[CSS] (광고)누구나 5천원 적립 스마일클럽이면 두배 적립! 마켓컬리 여름에도 늘 신선 풀콜드 샛별배송!
[CSS] [Y-DEC] 2019-2학기 동아리 홍보전
[CSS] 귀하를 위한 추천 강좌
[CSS] [창업 준비를 한번에!] 2019년 국제캠퍼스 학생벤처센터 창업 원스톱 상담창구 운영 안내
[CSS] ACS Pubs Forum 참가 및 포스터 발표 신청
[CSS] 연세대학교 통일연구원 Issue Brief 제129호
[CSS] The Spirit of Yonsei (섬김과 나눔, 기부와 헌신) - 7월호
[CSS] 연세소식(2019.07.15.) 함께 뛰며 나눔의 정신 나눠요
[CSS] [특별할인] 오늘 만료되는 우버이츠 할인을 놓치지마세요!
[CSS] 장기 미이용 휴면처리 완료 메일
[CSS] 이용 약관 업데이트
[CSS] 【アダルト動画】おすすめの作品をピックアップ★
[CSS] [특별할인] 우버이츠 첫 주문 1만원 할인 받으세요!
[CSS] 새로운 환경에서 로그인 되었습니다.
[CSS] [신한카드] 결제일별 이용기간 (신용공여기간) 변경 연기 안내
[CSS] [DBGuide.net] 2019 마이데이터 서비스 오픈 컨설팅 지원 안내
[CSS] (광고) POE, 랑그릿사, 일곱개의 대죄를 하는 사람이라면 주목!
[CSS] 最優秀女優賞👑相沢みなみの待望の新作💖【一番の痴女姿見せます💋】
[CSS] 2019년도 여름학기 2차 교내특강(토익, 토플) 개강 안내
[CSS] 새로운 기기(브라우저)에서 로그인 되었습니다.
[CSS] 【アダルト動画】本日更新セール情報★
[CSS] [DBGuide.net] 마이데이터 아이디어 발표회에 여러분을 초대합니다.(7/11)
[CS

In [201]:
driver.find_elements_by_xpath("//div[@class='list_menu']")[3].click()

- 개발자도구에 Application보면 Cookies 생성되어있음.
- 로그인 후에 Cookies값들 가져와서 이용하는게 원래 셀레니옴의 목적임
- 렌더링 하는데 걸리는 시간도 오래걸림

In [3]:
import requests
session = requests.Session()

In [464]:
session.cookies

<RequestsCookieJar[]>

In [521]:
for cookie in driver.get_cookies():
    #print((cookie["name"],cookie["value"]))
    session.cookies.set(cookie["name"],cookie["value"])

In [522]:
session.cookies

<RequestsCookieJar[Cookie(version=0, name='JSESSIONID', value='0B288ED9286AA136E1489A0171D83501', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False), Cookie(version=0, name='NID_AUT', value='zvFDJhqiMXVqRaiLPidaeX40HzMnS2KafSmeH/mBZUrRpCkrw330LnVV3aCUqED/', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False), Cookie(version=0, name='NID_JKL', value='igc4zMJQL4Cz+7plg8Pjx5QQd8ICXGCHinpC9vjdp70=', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rf

In [523]:
html = session.get("https://mail.naver.com/#%7B%22fClass%22%3A%22list%22%2C%22oParameter%22%3A%7B%22page%22%3A1%2C%22sortField%22%3A1%2C%22sortType%22%3A0%2C%22folderSN%22%3A%224%22%2C%22type%22%3A%22%22%2C%22isUnread%22%3Afalse%7D%7D")

In [524]:
html.text # 스팸이라는 단어가 나오면 제대로 뜬것(쿠키값 전달.)

'\n\n\n<!DOCTYPE HTML>\n\n\t\n\t\n\t\n\t\n\t\n\t\t<html lang="ko">\n\t\n\n\n<head>\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge">\n\t<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n\t\n\t\t\n\t\t\n\t\t\t\n\t\t\t\t\n\t\t\t\t\t<link rel="stylesheet" type="text/css" href="/css/mail2_le.min.190717.css">\n\t\t\t\t\t<link rel="stylesheet" type="text/css" href="/css/mail_skin.min.190717.css">\n\t\t\t\t\n\t\t\t\t\n\t\t\t\n\t\t\n\t\n\n\t\n\t\t\n\t\t\t\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t\t<link rel="dns-prefetch" href="//ssl.pstatic.net" />\n\t\t\t\t\t<link rel="dns-prefetch" href="//lcs.naver.com" />\n\t\t\t\t\t<link rel="dns-prefetch" href="//cc.naver.com" />\n\t\t\t\t\t<link rel="dns-prefetch" href="//static.nid.naver.com" />\n\t\t\t\t\t<link rel="dns-prefetch" href="//phinf.pstatic.net" />\n\t\t\t\t\t<link rel="stylesheet" type="text/css" href="/css/mail_me.min.190717.css">\n\t\t\t\t\n\t\t\t\n\t\t\t<link rel="shortcut icon" href="/favicon.ico?2">\n\t\t\t\n\t\t\n\n\t\

In [525]:
from html import unescape
html = session.post('https://mail.naver.com/json/list/?page=1&sortField=1&sortType=0&folderSN=5&type=&isUnread=false&u=http94')
for _ in html.json()["mailData"]:
    print(unescape(_["subject"]))

가입첫입금15％ 매일첫입금10％ 가입쿠폰 2만지급[성인]


AJAX로 보내는 것은 무조건 url이 어딘가 숨어있다.

In [222]:
html = requests.post('https://mail.naver.com/json/list/?page=1&sortField=1&sortType=0&folderSN=0&type=&isUnread=false&u=http94')
html.text
# 그냥 request보내면 실패한다.

'{"Result":"FAIL","Message":"", "LoginStatus":"NOLOGIN"}'

In [250]:
from html import unescape
html = session.get('https://section.cafe.naver.com/cafe-home-api/v1/homepc?myCafeCount=30&articleCount=3&useMyCafeEvent=true')
                    #'https://section.cafe.naver.com/cafe-home-api/v1/cafes/join?page=2&perPage=10&recentUpdates=true'
for _ in html.json()["message"]["result"]["myCafe"]["cafes"]:
     print("[cafeName]",unescape(_["cafeName"]))

[cafeName] 수만휘-수능날만점시험지를휘날리자★수능공부과외편입유학
[cafeName] 전대모 (대학생,레포트,토익,족보,스펙,공모전,자격증,편입학)
[cafeName] 연세전공알리미(YDMC)
[cafeName] 포만한 수학 연구소
[cafeName] 대학솔루션카페(대학생,레포트,공모전,스펙,족보,인턴,자격증)
[cafeName] 솔루션마을[대학생교레포트스펙족보공모전자격증편입학인턴취업]
[cafeName] 네버 엔딩 원피스 :: 끝나지 않을 그들의 항해
[cafeName] 메이플거래카페 - 메이플스토리1,2 아이템거래 전문카페
[cafeName] 대학원 입학을 준비하는 사람들의 모임
[cafeName] C언어를 배우자
[cafeName] 윤성우의 프로그래밍 스터디그룹 [C/C++/자바/파이썬]
[cafeName] 빅토리 테니스 아카데미
[cafeName] 던파카페 -던전앤파이터 커뮤니티(Since 2005)
[cafeName] 황인영 영어 카페
[cafeName] 컴퍼니 오브 히어로즈 매니아 카페 COHMC
[cafeName] 수학클럽
[cafeName] AccessON(엑세스온,액세스온)
[cafeName] 텝스캠프★TEPS/텝스시험/텝스학원/영어듣기/공부/서울대/특목고
[cafeName] 피파온라인4 커뮤니티 투게더 ▷피커투◁
[cafeName] 퍼스트PC - 피파대리 , 지피방, 프리미엄PC방,집
[cafeName] 바이러스 제로 시즌 2
[cafeName] 연세대학교 외국어학당 영어특강
[cafeName] 『최강미드』미드감상,보기/미드자막/영드/영어공부/영어자료


In [248]:
for _ in driver.find_elements_by_xpath("\
//div[@class='user_mycafe_list']\
//div[@class='user_mycafe_info']/a/strong[@class='name']"):
    print(_.text)

수만휘-수능날만점시험지를휘날리자★수능공부과외편입유학
전대모 (대학생,레포트,토익,족보,스펙,공모전,자격증,편입학)
연세전공알리미(YDMC)
포만한 수학 연구소
대학솔루션카페(대학생,레포트,공모전,스펙,족보,인턴,자격증)
솔루션마을[대학생교레포트스펙족보공모전자격증편입학인턴취업]
네버 엔딩 원피스 :: 끝나지 않을 그들의 항해
메이플거래카페 - 메이플스토리1,2 아이템거래 전문카페
대학원 입학을 준비하는 사람들의 모임
C언어를 배우자


In [264]:
for _ in driver.find_elements_by_css_selector(
    ".user_mycafe_list .user_mycafe_info .name"):
    print('[Name]',_.text)

[Name] 수만휘-수능날만점시험지를휘날리자★수능공부과외편입유학
[Name] 전대모 (대학생,레포트,토익,족보,스펙,공모전,자격증,편입학)
[Name] 연세전공알리미(YDMC)
[Name] 포만한 수학 연구소
[Name] 대학솔루션카페(대학생,레포트,공모전,스펙,족보,인턴,자격증)
[Name] 솔루션마을[대학생교레포트스펙족보공모전자격증편입학인턴취업]
[Name] 네버 엔딩 원피스 :: 끝나지 않을 그들의 항해
[Name] 메이플거래카페 - 메이플스토리1,2 아이템거래 전문카페
[Name] 대학원 입학을 준비하는 사람들의 모임
[Name] C언어를 배우자


In [261]:
driver.find_elements_by_css_selector(".user_mycafe_list .user_mycafe_info")

'수만휘-수능날만점시험지를휘날리자★수능공부과외편입유학\n대표카페\n즐겨찾기 선택 시 등록됨\n새글 4,115'

In [228]:
html.text

'{"message":{"status":"200","error":{"code":"","msg":""},"result":{"myCafe":{"cafes":[{"cafeId":10197921,"cafeName":"수만휘-수능날만점시험지를휘날리자★수능공부과외편입유학","cafeUrl":"suhui","articleNewCounts":4118,"formattedArticleNewCounts":"4,118","cafeThumbnailPcUrl":"https://cafeptthumb-phinf.pstatic.net/MjAxNzExMjNfNzMg/MDAxNTExNDE4MjU4MTg3.PbwPLlDNA1kIckG0yNlNCLJtIvWheKE5foy4j7juHf8g.WNoWukxLa48Z0k79s8dFPYJv-bZKjKol7adsbncXwe4g.JPEG.hansolmi/sumanhui_icon2.jpg?type=f72_72_mask","cafeThumbnailMobileUrl":"https://mcafethumb-phinf.pstatic.net/MjAxNzExMjNfNzMg/MDAxNTExNDE4MjU4MTg3.PbwPLlDNA1kIckG0yNlNCLJtIvWheKE5foy4j7juHf8g.WNoWukxLa48Z0k79s8dFPYJv-bZKjKol7adsbncXwe4g.JPEG.hansolmi/sumanhui_icon2.jpg?type=f138_138_mask","manageCafe":false,"dormantCafe":false,"powerCafe":true,"starJoinCafe":false,"gameCafe":false,"educationCafe":false,"mobileCafeName":"수만휘","aheadOfTime":"24초 전","lastUpdateDate":"2019-07-17 13:34:58","lastVisitDate":"2019-07-11 23:48:11","openType":"O","favoriteCafe":false,"hasNewArticle":tr

In [247]:
html.json()["message"]["result"]["myCafe"]['cafes']

[{'cafeId': 10197921,
  'cafeName': '수만휘-수능날만점시험지를휘날리자★수능공부과외편입유학',
  'cafeUrl': 'suhui',
  'articleNewCounts': 4124,
  'formattedArticleNewCounts': '4,124',
  'cafeThumbnailPcUrl': 'https://cafeptthumb-phinf.pstatic.net/MjAxNzExMjNfNzMg/MDAxNTExNDE4MjU4MTg3.PbwPLlDNA1kIckG0yNlNCLJtIvWheKE5foy4j7juHf8g.WNoWukxLa48Z0k79s8dFPYJv-bZKjKol7adsbncXwe4g.JPEG.hansolmi/sumanhui_icon2.jpg?type=f72_72_mask',
  'cafeThumbnailMobileUrl': 'https://mcafethumb-phinf.pstatic.net/MjAxNzExMjNfNzMg/MDAxNTExNDE4MjU4MTg3.PbwPLlDNA1kIckG0yNlNCLJtIvWheKE5foy4j7juHf8g.WNoWukxLa48Z0k79s8dFPYJv-bZKjKol7adsbncXwe4g.JPEG.hansolmi/sumanhui_icon2.jpg?type=f138_138_mask',
  'manageCafe': False,
  'dormantCafe': False,
  'powerCafe': True,
  'starJoinCafe': False,
  'gameCafe': False,
  'educationCafe': False,
  'mobileCafeName': '수만휘',
  'aheadOfTime': '13초 전',
  'lastUpdateDate': '2019-07-17 13:39:11',
  'lastVisitDate': '2019-07-11 23:48:11',
  'openType': 'O',
  'favoriteCafe': False,
  'hasNewArticle': True,
  'a

In [328]:
# 로그아웃 하는 방법
driver.find_element_by_css_selector("#gnb_name2 + a.gnb_btn_login").click()
# 눈에 안보이는 것을 클릭하는 것 자체가 말이 안되서 오류가 뜨는 것임.
# 일반 페이지들도 페이지가 로딩이 안되면 이런 에러가 뜬다.
# 이런 경우는 보일 때까지 기다려 줘야함.

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=75.0.3770.142)


In [515]:
dom = BeautifulSoup(driver.page_source,"lxml")
dom.select("#gnb_name2 + a.gnb_btn_login")
# dom에서는 정상임.

[<a class="gnb_btn_login" href="https://nid.naver.com/nidlogin.logout?returl=http://www.naver.com" id="gnb_logout_button"><span class="gnb_bg"></span><span class="gnb_bdr"></span><span class="gnb_txt">로그아웃</span></a>]

In [11]:
N = len([_ for _ in driver.find_elements_by_css_selector(
".hdline_news .hdline_article_tit >a, .com_list li >a")
     if len(_.get_attribute('href')) > 1])
N # 뉴스 갯수

35

In [12]:
driver.switch_to_window(driver.window_handles[0])

C:\Users\SeoDongUk\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.


In [10]:
# 새창 띄워서 여닫으면서 받아오기
for news in [_ for _ in driver.find_elements_by_css_selector(
                ".hdline_news .hdline_article_tit >a, .com_list li >a")
                    if len(_.get_attribute('href')) > 1]:
        print(news.text)
        newsID = news.get_attribute('href')
        driver.execute_script("window.open('about:blank');")
        driver.switch_to_window(driver.window_handles[-1])
        driver.get(newsID)
        print(driver.find_element_by_css_selector('#articleBodyContents').text)
        
        driver.close()
        driver.switch_to_window(driver.window_handles[0])

경찰, 양현석 '성매매 알선' 혐의로 입건 본격 수사


C:\Users\SeoDongUk\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use driver.switch_to.window instead
  


동영상 뉴스
[앵커]
성 접대 의혹으로 참고인 조사를 받았던 YG엔터테인먼트 양현석 전 대표가 피의자 신분으로 경찰의 수사를 받게 됐습니다.

경찰은 그동안 조사 내용을 분석해 양 씨를 성매매 알선 혐의로 오늘 입건했다고 밝혔습니다.

천상규 기자의 보도입니다.

[기자]
지난달 26일 검은 색 승용차를 타고 경찰에 출석해 9시간 동안 참고인 조사를 받은 양현석 전 YG 대표.

양 전 대표는 경찰에서 지난 2014년 외국인 재력가들을 상대로 성 접대를 했다는 의혹과 같은 해 이들의 유럽 여행에 유흥업소 여성들을 동원했다는 의혹에 대해 조사를 받았습니다.

양 전 대표는 당시 경찰 조사에서 지인의 초대로 식사만 했을 뿐, 성매매는 없었다며 모든 의혹을
강하게 부인했습니다.

그러나 경찰은 양 전 대표와 YG 직원의 2014년 지출 내역을 파악한 결과 양 전 대표가 성 접대한 것으로 보이는 정황을 포착했습니다.

이에 따라 경찰은 양 전 대표와 유흥업소 종사자 등 2명을 참고인 신분에서 피의자 신분으로 전환해 성매매 알선 혐의로 입건했습니다.

또 경찰은 2014년 유럽 여행에 동행했던 여성 가운데 일부로부터 성매매를 인정하는 취지의 진술을 확보했습니다.

이에 따라 유럽에서 접대를 받은 외국인 재력가 한 명과 유흥업소 여성 등 2명도 성매매 혐의로 입건했습니다.

경찰은 양 전 대표 등 4명이 입건됨에 따라 그동안 지지부진했던 수사가 활기를 띨 것으로 전망했습니다.

YTN 천상규입니다.

▶ 대한민국 24시간 뉴스 채널 YTN 생방송보기
▶ 네이버에서 YTN 뉴스 채널 구독하기


[저작권자(c) YTN & YTN PLUS 무단전재 및 재배포 금지]
"강제징용 손배청구권 남아있다" 입장밝힌 靑…논란 예고


C:\Users\SeoDongUk\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: use driver.switch_to.window instead
  del sys.path[0]


"2005년 민관위 조사서
불법행위 청구권소멸
결론 낸 적 없어" 반박

"日과 타협여지 줄여
논란키울 소지" 비판도

◆ 한일 정면충돌 ◆

강경화 외교부 장관(오른쪽)이 17일 오후 서울 종로 외교부 청사를 방문한 데이비드 스틸웰 신임 미국 국무부 동아태 차관보와 악수하고 있다. 가운데는 해리 해리스 주한 미국대사. [사진출처 = 연합뉴스]
청와대가 논란을 빚고 있는 일제시대 강제징용 피해자 개인의 손해배상청구권이 남아 있다는 취지로 사실상 유권 해석을 내렸다.

문재인 대통령과 청와대는 지난해 10월 대법원의 강제징용 판결 후 기회가 있을 때마다 3권 분립 원칙을 강조하며 정부가 관여할 수 없다는 입장을 거듭 강조해왔다. 문 대통령은 올해 1월 10일 신년기자회견에서 "일본 정부가 조금 더 겸허한 입장을 가져야 한다"면서 "일본 정치 지도자들이 그 문제(강제징용 판결)를 정치 쟁점화해서 논란거리로 만들고 확산시키는 것은 현명한 태도가 아니다"고 말했다.

개인의 손해배상청구권이 유효하게 남아 있는지에 대해서는 행정부 차원의 해석을 경계했지만 이번에 명확하게 정의를 내렸다는 점에서 주목된다.

청와대가 근거로 삼은 것은 한일회담 문서공개 후속대책 관련 2005년 민관공동위원회 활동 결과다. 당시 공식 발표 자료에 따르면 한일협상 당시 한국 정부는 일본 정부가 강제동원의 법적 배상·보상을 인정하지 않음에 따라 '고통받은 역사적 피해사실'에 근거하여 정치적 차원에서 '보상'을 요구했으며, 이러한 요구가 양국 간 무상자금산정에 반영되었다고 보아야 한다고 밝혔다. 이날 청와대의 해명 과정에서 드러난 유권 해석의 핵심은 '보상' 성격과 달리 위자료 등 '배상' 성격의 청구권은 여전히 존재한다는 뜻이다.

고민정 청와대 대변인은 17일 "당시 민관공동위에서 강제징용 피해자 개인의 손해배상청구권이 소멸되었다고 발표한 바 없다"고 밝혔다.

고 대변인은 "당시 민관공동위는 '한일 청구권협정은 한일 양국 간 재정적·민사적 채권·채무관계를 해결하기 위한 것이었을 뿐 반인도적

‘분당 수순’ 평화당·‘술렁’ 바른미래…제3지대 정계개편 신호탄
[서울신문]
유성엽·박지원 등 10명 대안정치 결성
정동영 사퇴·비대위 거부에 실력행사
반당권파 절반 넘어 당장은 탈당 안 해
“9월말 신당 출범… 3단계 로드맵 추진”
바른미래당 호남계 영입 물밑접촉 중

정 대표 “분열 선동하는 원로정치인”
민주평화당 정동영(가운데) 대표가 17일 오전 국회에서 열린 최고위원회의에서 발언하고 있다. 유성엽 원내대표 등 반당권파 최고위원들은 불참했다. 2019.7.17 연합뉴스
민주평화당이 분당 수순에 들어간 형국이다. 내년 총선을 9개월 앞두고 평화당이 쏘아 올린 정계개편 신호탄이 바른미래당 등을 흔들면서 제3지대 신당 창당 등 정계개편으로 이어질지 주목된다.

유성엽 원내대표와 박지원·천정배 의원 등 평화당 현역 의원 10명은 17일 ‘변화와 희망의 대안정치연대’(대안정치)를 결성했다. 대안정치에는 김종회·박지원·유성엽·윤영일·이용주·장병완·장정숙·정인화·천정배·최경환 의원이 이름을 올렸다. 현역 의원 16명(당적 기준 14명)이 활동하는 평화당의 다수다.

이들은 전날 심야 의총에서 정동영 대표를 필두로 한 당권파가 정 대표 사퇴와 비상대책위원회 체제 전환을 거부하자 설득 작업을 중단하고 실력행사에 나섰다.

유 원내대표는 이날 국회에서 기자간담회를 열고 “내년 총선에서 1당이 될 것”이라며 신당 창당 로드맵을 총 3단계로 설명했다. 그는 “가급적 신당이 9월 말에 출범했으면 한다”며 “정기국회가 끝난 12월과 내년 1월 2단계 변화를 하고, 총선에 임박해 3단계에 들어갈 것”이라고 했다. 그러면서 “우리가 1∼2단계를 잘 밟으면 3단계에서 대통령을 하고자 하는 분들이 ‘제3지대 신당에 가서 깃발을 들어야 대통령에 당선되겠구나’ 하고 우리에게 같이 하자고 사정할 사람들이 나올지 모른다”고 했다.

이들은 반당권파가 절반을 넘는 만큼 당장은 탈당하지 않고 신당 창당을 준비할 방침이다. 다만 대안정치 관계자는 “정 대표와 한순간도 함께할 수 없다며 당장 탈당을

동영상 뉴스
[앵커]
강남 유명 클럽 아레나처럼 거액의 탈세가 벌어진 유흥업소 등에 대해 국세청이 동시 세무조사에 들어갔습니다.

모바일 결제를 이용하거나 가상공간에 장부를 보관하는 등 탈세 수법이 첨단기술을 동원해 더욱 교묘해진 것으로 드러났습니다.

김평정 기자입니다.

[기자]
서울 강남의 유명 클럽 아레나는 지난해 국세청 세무조사에서 150억 규모의 탈세 혐의가 드러났습니다.

아레나처럼 적게는 수십억에서 많게는 수백억씩 탈세를 일삼은 유흥업소 등이 세무조사에서 무더기로 드러났습니다.

적발 대상에는 고급 유흥업소가 건물 층마다 친인척과 종업원의 명의로 따로 등록해 90억여 원을 탈세한 경우가 있었고,

여성 종업원 수백 명을 고용한 유흥업소가 수시로 친인척 명의로 개업과 폐업을 반복하면서 4백억 원가량 세금을 빼돌린 일도 있었습니다.

손님을 유치하는 영업직원인 MD를 이용해, SNS 등으로 모집한 손님에게 모바일 결제나 현금 입금을 유도하고 자연스럽게 신용카드 결제 기록을 남기지 않는 수법도 동원됐습니다.

함께 적발된 불법 대부업체는 종이 장부 대신 가상의 저장공간인 '클라우드'에 장부를 숨기기도 했습니다.

밀수한 니코틴 원액으로 액상 전자담배를 불법으로 만들고 팔아 탈세는 물론 국민 건강에도 해를 끼친 업체도 있었고,

겨우 2살 된 아기의 계좌로 수강료를 받고 현금영수증을 발급 안 한 영어학원도 적발 대상에 포함됐습니다.

국세청은 유흥업소와 대부업체에 대해서는 세무조사 후 고발하는 게 아니라 처음부터 처벌을 목적으로 한 조세범칙조사를 하기로 했습니다.

[이준오 / 국세청 조사국장 : 명의위장 유흥업소, 불법 대부업자의 경우 검찰과의 협의 채널을 최대한 가동하여 처음부터 압수수색 영장을 발부받아 조세범칙조사로 착수했습니다.]

이번 조사와는 별개로 국세청은 클럽 아레나와 버닝썬 논란을 계기로 전국 21개 유흥업소에 대해서도 동시 세무조사를 벌여 현재 마무리 단계에 있다고 밝혔습니다.

YTN 김평정[pyung@ytn.co.kr]입니다.

▶

화재감지기와 똑같은 몰카...비밀번호 알아내 절도
동영상 뉴스
[앵커]
'몰래카메라'는 크기나 모양이 다양해서 범죄에 악용될 때가 많습니다.

아파트 복도에 화재감지기 모양의 몰래카메라를 설치해 현관 비밀번호를 알아낸 절도범이 경찰에 붙잡혔습니다.

오태인 기자입니다.

[기자]
헬멧을 쓴 남성이 아파트 현관 잠금장치 쪽으로 카메라를 맞춘 뒤 계단을 내려갑니다.

하루 뒤 이 남성은 모자를 눌러쓰고 사다리를 든 채 다시 아파트에 나타납니다.

다시 나흘 뒤, 또 나타난 남성은 무언가에 놀란 듯 황급히 달아납니다.

살지도 않는 아파트에 수시로 나타난 이 남성은 36살 김 모 씨.

몰래카메라를 설치해 현관 비밀번호를 알아낸 뒤 돈을 훔쳤습니다.

[아파트 관계자 : 몰카를 설치했으니까 번호를 알았을 것 아닙니까? 번호를 알고 번호를 누르고 들어가다….]

김 씨가 설치한 몰래카메라는 화재감지기와 똑같은 모양입니다.

따라서 주민들은 몰래카메라를 전혀 눈치채지 못했습니다.

이 카메라로 주인이 문을 여는 모습을 찍은 뒤 비밀번호를 알아내 집으로 들어갈 수 있었습니다.

김 씨는 아파트만 턴 게 아니었습니다.

경남 거제에서는 금은방도 털었습니다.

그런데 두 범행 모두 같은 오토바이를 탔다가 덜미가 잡혔습니다.

[이승규 / 경남 마산동부경찰서 형사과장 : 피의자가 아파트에서 범행할 때도 오토바이를 사용했고 거제 금은방 절도 사건도 오토바이를 사용했는데 그 오토바이가 같은 오토바이로 확인됐습니다. 그래서 같은 피의자로 판단하고 추적하게 된 겁니다.]

경찰은 특수 절도 혐의로 김 씨의 구속영장을 신청했습니다.

YTN 오태인[otaien@ytn.co.kr]입니다.

▶ 대한민국 24시간 뉴스 채널 YTN 생방송보기
▶ 네이버에서 YTN 뉴스 채널 구독하기


[저작권자(c) YTN & YTN PLUS 무단전재 및 재배포 금지]
포항제철소에서 또 안전사고..."비용 절감 탓"
동영상 뉴스
[앵커]
포스코 포항제철소에서 또 안전사고가 발생했습니다.

협력업체 직원이

동영상 뉴스

훈민정음 해례본 상주본의 소유권이 국가에 있다는 대법원 판결 이후 처음으로 문화재청이 소장자 배익기 씨를 만났습니다.

배씨는 상주본을 돌려달라는 요구에 응하지 않았습니다.

1000억 원 정도를 주지 않으면 상주본을 돌려주지 않겠다는 입장을 바꾸지 않은 것입니다.

배씨는 상주본이 국가소유라는 판결에도 불구하고 적당한 보상을 통한 반환을 고집하고 있는데, 상주본의 소유권 무효 확인 소송 등 법적 대응을 검토하겠다고 했습니다.

이와 함께 현재 서울에서 활동하는 독지가에게 국가에 귀속하는 조건으로 1000억원 정도를 받고 상주본을 넘기는 방안도 논의 하고 있다고 밝혔습니다.

강나현(kang.nahyun@jtbc.co.kr)

▶ JTBC 뉴스 제보하기 ▶ 놓친 '뉴스룸' 다시보기

▶ 카카오톡 친구들에게 'JTBC를 선물'하세요!

Copyright by JTBC(http://jtbc.joins.com) and JTBC Content Hub Co., Ltd. All Rights Reserved. 무단 전재 및 재배포 금지
한국 유학생 강도 살해 '무기수' 27년만에 무죄 석방
홀맨 체스터 (방송 캡쳐) © 뉴스1

(서울=뉴스1) 김서연 기자 = 미국의 한 무기수가 다른 진범 가능성에 수감 27년만에 풀려났다고 CNN 등 현지언론들이 17일(현지시간) 보도했다.

CNN에 따르면 지난 1991년 필라델피아에서 한인 유학생 호태정씨를 강도 살해한 범인으로 몰린 체스터 홀맨 3세(48)가 수감중이던 펜실베이니아 루저네 주 교도소에서 16일 풀려났다. 앞서 필라델피아 민사소송법원 그웬돌린 브라이트 판사는 홀맨이 사건과 '결백한 것으로 보인다'고 이날 판결했다.

1967년 서울 출생으로 당시 펜실베이니아주립대 랭귀지코스를 다니던 호태정씨는 친구와 길을 걷던 중 2명의 강도로부터 변을 당했다. 목격자들에 따르면 호태정씨를 바닥에 눕힌 범인은 총격을 가한후 하얀색 쉐비 브레이저 SUV를 타고 달아났다.

911 수배 지령을 접한 경찰은 사건 4분후께 용

남부 최고 150mm 장맛비…태풍 ‘다나스’도 북상
[앵커]

장마전선이 다시 북상하고 있습니다.

호남, 경남지역에 내일(18일)까지 최고 150mm 넘는 많은 비가 예보됐습니다.

5호 태풍 '다나스'도 우리나라를 향해 북상하고 있습니다.

재난방송 스튜디오, 신방실 기상전문기자 연결돼 있습니다.

지금 장맛비는 어느 지역에 내리고 있나요?

[기자]

네, 현재 위성 영상인데요,

거대한 비구름을 동반한 장마전선이 남해안까지 북상했습니다.

그 영향으로 제주와 전남에 이어 다른 남부 지역에도 비가 시작됐고요.

밤새 충청과 경기, 강원 남부까지 확대되겠습니다.

이번 장맛비의 최대 고비는 오늘(17일) 밤부터 내일(18일) 오후 사이인데요.

남부의 붉은색으로 보이는 곳엔 시간당 30mm 이상의 폭우가 쏟아질 것으로 보입니다.

내일(18일)까지 예상 강우량은 호남과 경남은 최대 150mm, 제주와 경북은 30에서 80mm입니다.

남부지방엔 이번 장마 기간에 이미 400mm가량 비가 내렸기 때문에 산사태 등 피해도 우려됩니다.

[앵커]

태풍도 북상중이라고요,

그럼 장마전선과 겹치게될 것 같은데, 그럼 어떻게 되는건지, 어떤 영향을 주게되나요?

[기자]

네, 처음 보여드린 위성사진에서 장마전선 아래쪽, 그러니까 필리핀 부근에 소용돌이 구름이 보이죠.

어제(16일) 오후 발생한 제5호 태풍 '다나스'입니다.

아직 약한 소형급이지만, 경로가 위협적입니다.

내일(18일)은 타이완, 모레(19일)는 중국을 지나 제주 부근까지 빠르게 북상하겠고, 부산 상륙 가능성도 있습니다.

오늘(17일) 오전까지만 해도 서해에 진입할 것으로 예상됐는데, 한나절 만에 남해로 진로가 바뀔 만큼 불확실성이 큰 상황인데요.

다른 나라 기상청들이 예상한 진로도 제각각입니다.

태풍의 세력 자체는 북상하면서 약해지겠지만, 장마와 겹쳤다는 게 문제입니다.

열대 바다의 수증기를 몰고와 장마전선의 비구름을 강화시킬 가능성이 높습니다.

그래서 태풍이 근접하는 금요일부터 다음

In [21]:
for i in range(N):
    driver.find_element_by_css_selector("#lnb > ul > li:first-child > a").click()
    newsID = [_ for _ in driver.find_elements_by_css_selector(
            ".hdline_news .hdline_article_tit >a, .com_list li >a")
            if len(_.get_attribute('href')) > 1][i].click()
    
    #a = newsID.text
    #newsID.click()
    #b = driver.find_element_by_css_selector('#articleBodyContents').text
    #driver.get('https://news.naver.com/')

In [ ]:
<selenium.webdriver.remote.webelement.WebElement (session="d7d4850df00897ad1820d07b5f4f6da3", element="3fcdc32e-9277-40c6-972b-7c72c27908f3")>
<selenium.webdriver.remote.webelement.WebElement (session="d7d4850df00897ad1820d07b5f4f6da3", element="8165c200-6237-4929-87a7-927eec39c27f")>
# 새 창을 열면 세션정보가 달라진다.

In [528]:
driver.implicitly_wait()
# implicit sleep : 암묵적으로 계속 기다림 정해진 초만큼 sleep과 동일
# explicit wait : 그 event가 무시되거나 나타날때까지 간격별로 계속 검사하는 것임.

TypeError: implicitly_wait() missing 1 required positional argument: 'time_to_wait'

In [546]:
driver.find_element_by_css_selector("#gnb_name2 + a.gnb_btn_login").click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=75.0.3770.142)


In [536]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC


In [537]:
wait = WebDriverWait(driver,10,0.5
                     ,[ElementNotVisibleException])

In [538]:
wait.until(
    EC.visibility_of_element_located(
        (By.CSS_SELECTOR,"#gnb_name2 + a.gnb_btn_login")))
driver.find_element_by_css_selector("#gnb_name2 + a.gnb_btn_login").click()

# 뉴스기사는 elements나 all element를 쓰면 된다.

In [542]:
wait = WebDriverWait(driver,10,0.5
                     ,[ElementNotVisibleException])
wait.until(
lambda x:x.find_element_by_css_selector(
    "#gnb_name2 + a.gnb_btn_login").is_displayed())
driver.find_element_by_css_selector("#gnb_name2 + a.gnb_btn_login").click()

In [539]:
logout = driver.find_element_by_css_selector("#gnb_name2 + a.gnb_btn_login")

In [541]:
logout.is_displayed()

False

In [589]:
iframe = driver.find_element_by_name("da_iframe_time") # 그 안의 dom으로 다시 간다.
driver.switch_to_frame(iframe)

C:\Users\SeoDongUk\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: use driver.switch_to.frame instead
  


In [590]:
driver.find_element_by_tag_name("img").get_attribute("src")

'https://ssl.pstatic.net/tveta/libs/1242/1242461/f4c778fe28e51ecc6ee2_20190715175440120.jpg'

In [592]:
driver.switch_to_default_content()

C:\Users\SeoDongUk\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.default_content instead
  """Entry point for launching an IPython kernel.


In [670]:
driver.window_handles # window이름임.
driver.switch_to_window(driver.window_handles[-1])

C:\Users\SeoDongUk\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: use driver.switch_to.window instead
  


In [671]:
driver.window_handles # window이름임.

['CDwindow-6B1CB98F5C1418C47E17BDA2BB662237']

In [631]:
driver.find_element_by_id("userId").send_keys("test")

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=75.0.3770.142)


In [28]:
driver.get("http://www.kt.com")

In [29]:
driver.find_element_by_css_selector("span.quick > a").click()

In [661]:
driver.close()

In [634]:
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="cdc2692ee1a48775e1924bcb505866fb")>

In [636]:
# new Window 띄우기 : 단축키나 
from selenium.webdriver.common.keys import Keys

In [639]:
Keys.CONTROL + 't' # 새 탭
Keys.CONTROL + 'n' #새 창

'\ue009n'

In [ ]:
# 어차피 정보는 Selenium으로 받지 말라고 했는데 왜 click되는거 기다리는 걸 하는건지?

In [650]:
driver.find_element_by_tag_name("body").send_keys(Keys.chord(Keys.CONTROL+"N"))

In [652]:
Keys.(Keys.CONTROL+"N")

TypeError: 'str' object is not callable

In [665]:
driver.execute_script("javascript:window.open('about:blank');")

In [672]:
driver.get("https://www.google.com")

In [681]:
driver.execute_script("javascript:console.log('%s');")